<a href="https://colab.research.google.com/github/pat-olhed/Store_Forecast_TSA/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io
import gdown

In [27]:
!pip install darts

# Load Data

In [28]:
# Google Drive Mounten
def make_drive_url(file_id):
    return f"https://drive.google.com/uc?id={file_id}"

# Helper function to load a CSV from a direct URL
def load_csv_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request fails
    return pd.read_csv(io.StringIO(response.text))

def read_metadata_files(file_ids):
  # Read all files but train.csv
  # Load each CSV using the helper functions
  df_holiday_events = load_csv_from_url(make_drive_url(file_ids["holiday_events"]))
  df_items          = load_csv_from_url(make_drive_url(file_ids["items"]))
  df_oil            = load_csv_from_url(make_drive_url(file_ids["oil"]))
  df_stores         = load_csv_from_url(make_drive_url(file_ids["stores"]))
  df_transactions   = load_csv_from_url(make_drive_url(file_ids["transactions"]))
  return df_holiday_events, df_items, df_oil, df_stores, df_transactions

# Dictionary of file IDs
file_ids = {
    "holiday_events": "1RMjSuqHXHTwAw_PGD5XVjhA3agaAGHDH",
    "items": "1ogMRixVhNY6XOJtIRtkRllyOyzw1nqya",
    "oil": "1Q59vk2v4WQ-Rpc9t2nqHcsZM3QWGFje_",
    "stores": "1Ei0MUXmNhmOcmrlPad8oklnFEDM95cDi",
    "train": "1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv",
    "transactions": "1PW5LnAEAiL43fI5CRDn_h6pgDG5rtBW_"
}

df_holiday_events, df_items, df_oil, df_stores, df_transactions = read_metadata_files(file_ids)

# Filtering the Data

## - Region Guayas

In [38]:
# Download the train.csv file using gdown
train_url = make_drive_url(file_ids["train"])
gdown.download(train_url, "train.csv", quiet=False)

# Get Guayas store IDs
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()

# Initialize an empty list to hold filtered chunks
filtered_chunks = []
chunk_size = 10 ** 6

# Read the CSV file in chunks
for chunk in pd.read_csv('./train.csv', chunksize=chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk

# Concatenate all filtered chunks into a single DataFrame
df_train = pd.concat(filtered_chunks, ignore_index=True)

# Clean up to free memory
del filtered_chunks

Downloading...
From (original): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv
From (redirected): https://drive.google.com/uc?id=1oEX8NEJPY7wPmSJ0n7lO1JUFYyZjFBRv&confirm=t&uuid=43c0f280-5b7a-4c22-970b-b3c1da49bdad
To: /content/train.csv

  0%|          | 0.00/5.00G [00:00<?, ?B/s]
  0%|          | 524k/5.00G [00:00<1:03:57, 1.30MB/s]
  0%|          | 1.05M/5.00G [00:00<37:25, 2.23MB/s] 
  0%|          | 2.10M/5.00G [00:00<19:34, 4.25MB/s]
  0%|          | 4.19M/5.00G [00:00<09:53, 8.41MB/s]
  0%|          | 6.29M/5.00G [00:00<08:32, 9.74MB/s]
  0%|          | 10.5M/5.00G [00:01<04:54, 16.9MB/s]
  0%|          | 14.2M/5.00G [00:01<03:53, 21.3MB/s]
  0%|          | 17.8M/5.00G [00:01<03:23, 24.5MB/s]
  0%|          | 22.0M/5.00G [00:01<02:56, 28.2MB/s]
  1%|          | 25.7M/5.00G [00:01<02:47, 29.7MB/s]
  1%|          | 29.4M/5.00G [00:01<02:41, 30.8MB/s]
  1%|          | 33.6M/5.00G [00:01<02:32, 32.5MB/s]
  1%|          | 37.2M/5.00G [00:01<02:31, 32.8MB/s]
  1%|   

In [35]:
df_train.shape

(22941656, 6)

## - Top 3 Products

In [39]:
# Compute the number of items per family and select the top 3 families
items_per_family = df_items['family'].value_counts().reset_index()
items_per_family.columns = ['Family', 'Item Count']
top_3_families = items_per_family.head(3)  # Keep only the top 3 families
item_ids = df_items[df_items['family'].isin(top_3_families['Family'].unique())]['item_nbr'].unique()

df_train = df_train[df_train['item_nbr'].isin(item_ids)]

df_train.shape

(14745768, 6)